In [1]:
ls

Answers/             lambda_assume_policy_doc.json
bucket_hint.txt      lambda_permissions_policy_doc.json
create_group.sh*     power_user_marketing.json
delete.sh*           power_user_marketing_not_action_policy_doc.json
flag                 Readme.md
kingme.py            recon.sh*
kingme.sh*           sample-env.sh
lab1_notebook.ipynb  setup.sh*


In [ ]:
# %load Readme.md
# LAB_1_Lambda_Privesc_via_bad_NotAction

This lab covers the following scenario:

* An attacker discovers user credentials for marketing-dave which allows lambda management.
* Due to a flaw in the policy intended to restrict marketing-dave's ability to modify IAM privileges, the attacker can priv-esc to admin by launching a lambda with high privileges.
* The attacker discovers a secret in an S3 bucket after granting themselves S3 privileges to modify bucket policies

## Part I: Diagnosing and Exploiting the Problem

This lab examines a real-word issue discovered on a cloud pen test.
Devops had wanted to restrict the marketing team from privilege escalation by disallowing `iam:*` actions.

The policy they used was as follows:

        {
            "Arn": "arn:aws:iam::111111111111:group/PowerUsers-marketing-group",
            "AttachedManagedPolicies": [
                {
                    "PolicyArn": "arn:aws:iam::aws:policy/AWSLambdaFullAccess",
                    "PolicyName": "AWSLambdaFullAccess"
                }
            ],
            "CreateDate": "2012-12-20T16:20:10+00:00",
            "GroupId": "AGPAJNZACMXXXXXXXXXX",
            "GroupName": "owerUsers-marketing-group",
            "GroupPolicyList": [
                {
                    "PolicyDocument": {
                        "Statement": [
                            {
                                "Effect": "Allow",
                                "NotAction": "iam:*",
                                "Resource": "*"
                            }
                        ]
                    },
                    "PolicyName": "PowerUserAccess-marketing-2012122000000"
                }
            ],
            "Path": "/"
        }

Notice the NotAction in the PolicyDocument. 
* What is this trying to do? 
* What does it actually do?

See the answer in [Answers/Readme_Theory.md](Answers/Readme_Theory.md)


We will explore this with a scenario where an attacker has obtained the credentials to a user `marketing-dave`.
who is part of the PowerUsers-marketing-group. We will demonstrate how the policy above can be abused to
create a lambda function with a high-privileged role to elevate dave's own privileges.


1. Using admin creds, Create the roles and policies to simulate discovered high-privilege .

    ./setup.sh

In [11]:
ls -alh /root/.aws

total 24K
drwxr-xr-x 6 root root  192 Jan 13 15:44 ./
drwx------ 1 root root 4.0K Apr 14 14:02 ../
-rw------- 1 root root 1.1K Jan 19 13:47 config
-rw------- 1 root root  12K Apr 12 17:11 credentials
-rw-r--r-- 1 root root 1.0K Jan 13 15:44 .credentials.swp
drwxr-xr-x 3 root root   96 Jan  8 21:49 sso/


In [12]:
!whoami

root


In [9]:
! ./setup.sh


		Be sure to delete everything afterwards by running the delete.sh script

Unable to locate credentials. You can configure credentials by running "aws configure".
Could not get AWS_ACCOUNT_ID. Check your creds with aws sts get-caller-identity


In [ ]:
2. Perform reconnaissance. Run with --admin if you want to simulate discovering high-privilege creds (capable of IAM introspection).
   More often, you won't be able to list the discocered credentials IAM privileges, but you can run enumerate-iam.py linked in the script.
   ./recon.sh [--amin]

In [ ]:
# %load recon.sh
set -x

source env.sh

echo "If you are not familiar with inline vs managed policies see https://docs.aws.amazon.com/IAM/latest/UserGuide/access_policies_managed-vs-inline.html"

if [ -z $1 ]; then
  echo "using low privilege keys found in keys.json"
  export AWS_ACCESS_KEY_ID=`cat keys.json | jq -r '.AccessKey.AccessKeyId'`
  export AWS_SECRET_ACCESS_KEY=`cat keys.json | jq -r '.AccessKey.SecretAccessKey'`
  echo "----------------- The following fail because our discovered user has limited IAM permissions --------------------"
else
  echo "Using high privilege keys"
fi

echo "Determine the identity of the discovered credentials"

In [7]:
!aws sts get-caller-identity

Unable to locate credentials. You can configure credentials by running "aws configure".


In [ ]:
user_arn=`aws sts get-caller-identity | jq '.Arn'`
user_name=`echo $user_arn | sed 's|\"||g' | cut -d '/' -f 2`

echo "See if any interesting inline policies are attached directly to the user."
aws iam list-user-policies --user-name $USER_NAME

echo "No luck. Next see if any managed policies are attached to the user."
aws iam list-attached-user-policies --user-name $USER_NAME

echo "Also, no luck. The user must be in a group which grants the privileges."
aws iam list-groups-for-user --user-name $USER_NAME
group_name=`aws iam list-groups-for-user --user-name $USER_NAME | jq '.Groups[0].GroupName'`

echo "First check for group inline policies"
aws iam list-group-policies --group-name group_name

echo "Still no luck! Now check for attached group policies".
aws iam list-attached-group-policies --group-name group_name

echo "At last! To see what permissions the policy has, use list-policy-versions (because managed policies can have many versions)"
echo "Then use get-policy-version to get the permissions using the latest or default version"

printf "\n\n------------------------- Automated recon with low-priv keys ---------------------------------\n\n"

echo "Run enumerage-iam.py --access-key --secret-key    according to https://github.com/praetorian-code/enumerate-iam"
echo "Skipping to the results which show that we have iam list-roles and iam lambda list-functions"

aws lambda list-functions --max-items 5 | jq '.Functions[].FunctionName'
aws iam list-roles --max-items 5 

# aws lambda list-functions | jq '.Functions[] | select(.FunctionName | test("kingme"))'


aws iam list-policy-versions --policy-arn arn:aws:iam::aws:policy/AWSLambdaFullAccess
aws iam get-policy-version --policy-arn arn:aws:iam::aws:policy/AWSLambdaFullAccess --version-id v8

echo "Notice that the policy includes iam:PassRole and the Resource is '*'"
echo "An experienced attacker will recognized that this is the Wildcard Passrole priv-esc to Admin."
echo "We have discovered two paths to privilege escalation."
echo "1. We can run arbitrary functions. If a lambda function has a high-privilege role attached and we can modify the code, we can gain any privilege in the attached role."
echo "2. Since our group allows Wildcard Passrole via the AWSLambdaFullAccess, we can create a lambda and attach any role that already exists."

In [ ]:
3. Using dave's credentials, create the lambda function using the `discovered_role_with_iam_privs` role.

   ./kingme.sh

4. Once you are king (IAM boss), you can give yourself S3 Full Access privileges. Then you can list buckets.
You will need to [change the bucket IAM policy](https://aws.amazon.com/blogs/security/how-to-restrict-amazon-s3-bucket-access-to-a-specific-iam-role/) 
to grant yourself access to the bucket.



## Part II: Fixing the Problem (WIP)

Now comes the hard part. How do we accomplish what the Devops team had intended? Allow the marketing power users as
much freedom as possible (at least as much control over Lambda as possible) without giving them a path to privilege escalate to Admin?

### Options

* Permissions Boundary
* Explicit Deny
* Whitelist Passrole Resources
* Separate AWS account where marketing is admin + CI/CD to publish into existing account



## TODO
- [x] Add cleanup instrutions.
- [x] Replace all hard-coded random sequences with $RANDOM or similar. 
- [] Change the group privileges to only allow assume-role and push all privs into that group role, following best practices.
- [] Design a version of the Lab so that the priv-esc is only to read a CTF hash in S3, not become admin.
- [] Develop the "How to Fix" options.

In [ ]:
./setup.sh